## Imports

In [1]:
# Import required packages
import skimage as sk

from skimage import io, exposure, feature, filters, morphology, color, restoration, util

from skimage.filters import rank, threshold_otsu, threshold_local
from skimage.morphology import disk, watershed
from skimage.feature import peak_local_max
from skimage.future import graph
from scipy import ndimage as ndi
from scipy.spatial import distance as dist
import scipy.cluster.hierarchy as hier

import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd

## Functions for selecting an image file and saving a dataframe to a CSV

In [2]:
# prompt for image

def get_image():
    img_file = input('Enter full file path: ')
    img_raw = sk.io.imread(img_file)
    return img_raw

def save_to_file(dataframe):
    file_name = input('Enter a file path to save to (do not include .csv):')
    file_name = file_name + ".csv"
    dataframe.to_csv(file_name)

## Visualization

In [3]:
# Function to show images. 
# Takes image file, number of planes of the image to be shown, and a tuple/list containing the desired shape 
# of the axes array (optional)

def show_image(image, nplanes, subplot_shape = None):
    if subplot_shape == None:
        for i in range(nplanes):
            plt.figure()
            plt.imshow(image[i,:,:], cmap = 'gray')
            plt.axis('off')
    else:
        f, ax = plt.subplots(subplot_shape[0], subplot_shape[1], figsize = (20,20))
        ax = ax.ravel()
        for i in range(nplanes):
            ax[i].imshow(img_raw[i,:,:], cmap = 'gray')
            ax[i].axis('off')
            ax_title = str(i+1)
            ax[i].set_title(ax_title)
            
def img_hist_single(image):

    f, ax = plt.subplots(ncols=2, figsize=(20,10));
    ax[0].hist(image.ravel(), bins=256)
    ax[1].imshow(image, cmap='gray')
    
def img_hist_stack(image, stack):

    f, ax = plt.subplots(ncols=2, figsize=(20,10));
    ax[0].hist(image[stack,:, :].ravel(), bins=256)
    ax[1].imshow(image[stack,:, :], cmap='gray') 
    
# convenience function for plotting detected blobs
def plot_blobs(blobs, color, size_mod, fill = True):
    for blob in blobs:
        y, x, size, z = blob
        c = plt.Circle((x, y), size*size_mod, color=color, linewidth=2, fill=fill)
        ax.add_patch(c)
        
def plot_cells_inplane(image, cells):
    """Convenience function for plotting the cells found in each plane on an image of that plane. 
    Takes image and cells. Cells must be pandas data frame with columns x, y, and z. Z must contain integers only."""
    for i in range(len(image)):
        dat = cells[cells['z'] == i]
        plt.figure()
        plt.imshow(image[i, :,:])
        plt.scatter(dat['x'], dat['y'], c='r')